In [5]:
import os
import subprocess
import platform
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageOps
import cv2
import numpy as np
from scipy.spatial import distance
import math
from datetime import datetime
import tempfile
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import gc  # For garbage collection

In [6]:
# Improved Registration Mark Detector and Frame Extractor Classes

class ImprovedRegistrationMarkDetector:
    """
    Enhanced registration mark detector that can handle various mark sizes, colors, and patterns.
    Designed to be robust against different printing conditions and scan qualities.
    """
    
    def __init__(self):
        # Configuration parameters with reasonable defaults
        self.min_mark_size = 5  # Minimum size of registration mark in pixels
        self.max_mark_size = 30  # Maximum size of registration mark in pixels
        self.mark_threshold = 0.6  # Default correlation threshold for template matching
        self.clustering_distance = 20  # Distance threshold for clustering duplicate detections
        self.debug_mode = False  # Whether to generate debug visualizations
        
    def set_parameters(self, min_size=None, max_size=None, threshold=None, 
                      clustering_dist=None, debug=None):
        """Set detector parameters."""
        if min_size is not None:
            self.min_mark_size = min_size
        if max_size is not None:
            self.max_mark_size = max_size
        if threshold is not None:
            self.mark_threshold = threshold
        if clustering_dist is not None:
            self.clustering_distance = clustering_dist
        if debug is not None:
            self.debug_mode = debug
    
    def detect_marks(self, image):
        """
        Main method to detect registration marks in an image.
        Uses multiple detection strategies and combines results.
        
        Args:
            image: Input image (BGR format from OpenCV)
            
        Returns:
            List of (x, y) coordinates of detected registration marks
        """
        # Check if image is valid
        if image is None or image.size == 0 or len(image.shape) < 2:
            print("Warning: Invalid input image for registration mark detection")
            return []
            
        # Create a copy for visualization if debug mode is on
        debug_img = image.copy() if self.debug_mode else None
        
        # Convert to different color spaces for robust detection
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Collect marks from multiple detection methods
        all_marks = []
        
        # Method 1: Template matching with various sizes
        template_marks = self._detect_with_templates(gray)
        all_marks.extend(template_marks)
        
        # Method 2: Color-based detection for specific colors (like magenta in RISO prints)
        color_marks = self._detect_with_color(hsv)
        all_marks.extend(color_marks)
        
        # Method 3: Contour-based detection
        contour_marks = self._detect_with_contours(gray)
        all_marks.extend(contour_marks)
        
        # Remove duplicates through clustering
        unique_marks = self._cluster_marks(all_marks)
        
        # Filter marks based on grid pattern analysis
        filtered_marks = self._filter_marks_by_grid_pattern(unique_marks)
        
        # Draw detected marks on debug image
        if self.debug_mode and debug_img is not None:
            for x, y in filtered_marks:
                cv2.circle(debug_img, (x, y), 5, (0, 255, 0), 2)
                cv2.drawMarker(debug_img, (x, y), (0, 0, 255), 
                              markerType=cv2.MARKER_CROSS, markerSize=10, thickness=1)
            
            # Save debug image
            cv2.imwrite("debug_registration_marks.png", debug_img)
            
        # Clean up to free memory
        del gray, hsv, all_marks, debug_img
        gc.collect()
        
        return filtered_marks
    
    def _detect_with_templates(self, gray_img):
        """
        Detect registration marks using template matching with multiple templates.
        
        Args:
            gray_img: Grayscale input image
            
        Returns:
            List of (x, y) coordinates of detected marks
        """
        marks = []
        
        # Generate templates of different sizes - limit to just a few sizes to save memory
        template_sizes = [self.min_mark_size, 
                         (self.min_mark_size + self.max_mark_size) // 2, 
                         self.max_mark_size]
        
        # Apply adaptive thresholding to handle different brightness/contrast
        binary = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY_INV, 21, 5)
        
        # Create and apply multiple templates
        for size in template_sizes:
            # Create crosshair template
            crosshair_template = np.zeros((size, size), dtype=np.uint8)
            cv2.line(crosshair_template, (0, size//2), (size, size//2), 255, 1)
            cv2.line(crosshair_template, (size//2, 0), (size//2, size), 255, 1)
            
            # Create crosshair with circle template
            circle_template = crosshair_template.copy()
            cv2.circle(circle_template, (size//2, size//2), size//4, 255, 1)
            
            # Apply template matching with both templates
            for template in [crosshair_template, circle_template]:
                result = cv2.matchTemplate(binary, template, cv2.TM_CCOEFF_NORMED)
                locations = np.where(result >= self.mark_threshold)
                
                # Limit the number of matches to prevent memory issues
                max_matches = 1000  # Set a reasonable limit
                locations_y = locations[0][:max_matches] if len(locations[0]) > max_matches else locations[0]
                locations_x = locations[1][:max_matches] if len(locations[1]) > max_matches else locations[1]
                
                for y, x in zip(locations_y, locations_x):
                    center_x = x + size // 2
                    center_y = y + size // 2
                    marks.append((center_x, center_y))
        
        return marks
    
    def _detect_with_color(self, hsv_img):
        """
        Detect registration marks based on color filtering.
        Particularly useful for RISO prints with specific colors.
        
        Args:
            hsv_img: HSV color space image
            
        Returns:
            List of (x, y) coordinates of detected marks
        """
        marks = []
        
        # Define color ranges for common registration mark colors
        # Magenta/pink (common in RISO prints)
        lower_magenta = np.array([140, 50, 100])
        upper_magenta = np.array([170, 255, 255])
        
        # Cyan (another common color)
        lower_cyan = np.array([85, 50, 100])
        upper_cyan = np.array([110, 255, 255])
        
        # Black (common for registration marks)
        lower_black = np.array([0, 0, 0])
        upper_black = np.array([180, 255, 50])
        
        color_ranges = [
            (lower_magenta, upper_magenta),
            (lower_cyan, upper_cyan),
            (lower_black, upper_black)
        ]
        
        for lower, upper in color_ranges:
            # Create mask for this color range
            mask = cv2.inRange(hsv_img, lower, upper)
            
            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, 
                                         cv2.CHAIN_APPROX_SIMPLE)
            
            # Limit the number of contours to process to prevent memory issues
            max_contours = 1000  # Set a reasonable limit
            contours = contours[:max_contours] if len(contours) > max_contours else contours
            
            for contour in contours:
                # Filter by size
                area = cv2.contourArea(contour)
                if area < 5 or area > 500:  # Skip too small or too large areas
                    continue
                
                # Get center of contour
                M = cv2.moments(contour)
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    marks.append((cx, cy))
        
        return marks
    
    def _detect_with_contours(self, gray_img):
        """
        Detect registration marks using contour analysis.
        
        Args:
            gray_img: Grayscale input image
            
        Returns:
            List of (x, y) coordinates of detected marks
        """
        marks = []
        
        # Apply different thresholding methods for robustness
        _, binary_otsu = cv2.threshold(gray_img, 0, 255, 
                                     cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Process binary image
        # Apply morphological operations to enhance registration marks
        kernel = np.ones((3, 3), np.uint8)
        binary = cv2.morphologyEx(binary_otsu, cv2.MORPH_OPEN, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_LIST, 
                                     cv2.CHAIN_APPROX_SIMPLE)
        
        # Limit the number of contours to process to prevent memory issues
        max_contours = 1000  # Set a reasonable limit
        contours = contours[:max_contours] if len(contours) > max_contours else contours
        
        for contour in contours:
            # Filter contours by area
            area = cv2.contourArea(contour)
            if area < 5 or area > 500:
                continue
            
            # Check if contour has approximately 4 corners (crosshair shape)
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
            
            # Registration marks often have specific shapes
            if len(approx) >= 4 and len(approx) <= 8:
                # Get center of contour
                M = cv2.moments(contour)
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    marks.append((cx, cy))
        
        return marks
    
    def _cluster_marks(self, marks):
        """
        Remove duplicate detections by clustering nearby points.
        
        Args:
            marks: List of (x, y) coordinates
            
        Returns:
            List of unique (x, y) coordinates
        """
        if not marks:
            return []
        
        # Limit the number of marks to process to prevent memory issues
        max_marks = 2000  # Set a reasonable limit
        if len(marks) > max_marks:
            print(f"Warning: Limiting marks from {len(marks)} to {max_marks} to prevent memory issues")
            marks = marks[:max_marks]
        
        # Convert to numpy array
        points = np.array(marks)
        
        # Use DBSCAN clustering to group nearby points
        # Set a reasonable eps value based on the clustering_distance
        clustering = DBSCAN(eps=self.clustering_distance, min_samples=1).fit(points)
        
        # Get cluster centers
        unique_marks = []
        # Get unique cluster labels (should be a small number)
        unique_labels = set(clustering.labels_)
        # Limit the number of clusters to process if there are too many
        max_clusters = 500
        if len(unique_labels) > max_clusters:
            print(f"Warning: Limiting clusters from {len(unique_labels)} to {max_clusters}")
            unique_labels = list(unique_labels)[:max_clusters]
            
        for cluster_id in unique_labels:
            # Get points in this cluster
            cluster_mask = clustering.labels_ == cluster_id
            cluster_points = points[cluster_mask]
            
            # Calculate center of cluster
            center_x = int(np.mean(cluster_points[:, 0]))
            center_y = int(np.mean(cluster_points[:, 1]))
            unique_marks.append((center_x, center_y))
        
        return unique_marks
    
    def _filter_marks_by_grid_pattern(self, marks):
        """
        Filter marks based on grid pattern analysis.
        Removes outliers that don't fit the expected grid pattern.
        
        Args:
            marks: List of (x, y) coordinates
            
        Returns:
            Filtered list of (x, y) coordinates
        """
        if len(marks) < 4:  # Need at least 4 marks to form a grid
            return marks
        
        # Convert to numpy array
        points = np.array(marks)
        
        # Calculate pairwise distances
        dist_matrix = distance.cdist(points, points)
        
        # Find minimum non-zero distance for each point
        min_distances = []
        for i in range(len(points)):
            # Get all distances except self (zero)
            distances = dist_matrix[i]
            distances = distances[distances > 0]
            if len(distances) > 0:
                min_distances.append(np.min(distances))
        
        # Calculate median of minimum distances
        if min_distances:
            median_min_dist = np.median(min_distances)
            
            # Filter points that have at least one neighbor within reasonable distance
            filtered_marks = []
            for i, point in enumerate(points):
                # Check if this point has at least one neighbor within 2x median distance
                has_neighbor = np.any((dist_matrix[i] > 0) & 
                                     (dist_matrix[i] < 2 * median_min_dist))
                if has_neighbor:
                    filtered_marks.append((point[0], point[1]))
            
            return filtered_marks
        
        return marks  # Return original if we couldn't filter

    def analyze_grid_structure(self, marks, img_width, img_height):
        """
        Analyze the grid structure of detected marks to determine rows and columns.
        
        Args:
            marks: List of (x, y) coordinates
            img_width: Width of the image
            img_height: Height of the image
            
        Returns:
            Dictionary with grid analysis results including:
            - rows: List of mark rows
            - columns: List of mark columns
            - estimated_frames_per_row: Estimated number of frames per row
            - estimated_rows: Estimated number of rows
        """
        if len(marks) < 4:
            return {
                "rows": [],
                "columns": [],
                "estimated_frames_per_row": 0,
                "estimated_rows": 0
            }
        
        # Sort marks by y-coordinate
        sorted_by_y = sorted(marks, key=lambda m: m[1])
        
        # Adaptive tolerance based on image size and number of marks
        y_tolerance = img_height * 0.02  # 2% of image height
        
        # Group marks into rows
        rows = []
        current_row = [sorted_by_y[0]]
        
        for i in range(1, len(sorted_by_y)):
            if abs(sorted_by_y[i][1] - current_row[0][1]) < y_tolerance:
                # Same row
                current_row.append(sorted_by_y[i])
            else:
                # New row
                rows.append(sorted(current_row, key=lambda m: m[0]))  # Sort row by x-coordinate
                current_row = [sorted_by_y[i]]
        
        # Add the last row
        if current_row:
            rows.append(sorted(current_row, key=lambda m: m[0]))
        
        # Sort marks by x-coordinate
        sorted_by_x = sorted(marks, key=lambda m: m[0])
        
        # Adaptive tolerance based on image size
        x_tolerance = img_width * 0.02  # 2% of image width
        
        # Group marks into columns
        columns = []
        current_col = [sorted_by_x[0]]
        
        for i in range(1, len(sorted_by_x)):
            if abs(sorted_by_x[i][0] - current_col[0][0]) < x_tolerance:
                # Same column
                current_col.append(sorted_by_x[i])
            else:
                # New column
                columns.append(sorted(current_col, key=lambda m: m[1]))  # Sort column by y-coordinate
                current_col = [sorted_by_x[i]]
        
        # Add the last column
        if current_col:
            columns.append(sorted(current_col, key=lambda m: m[1]))
        
        # Estimate frames per row based on column count
        # Each frame has registration marks on both sides, so frames = columns - 1
        estimated_frames_per_row = max(1, len(columns) - 1)
        
        # Estimate number of rows based on row count
        # Each frame has registration marks on top and bottom, so frames = rows - 1
        estimated_rows = max(1, len(rows) - 1)
        
        return {
            "rows": rows,
            "columns": columns,
            "estimated_frames_per_row": estimated_frames_per_row,
            "estimated_rows": estimated_rows
        }

In [7]:
class FrameExtractor:
    """
    Enhanced frame extractor that uses the improved registration mark detector
    to accurately extract frames from contact sheets.
    """
    
    def __init__(self):
        self.mark_detector = ImprovedRegistrationMarkDetector()
        self.debug_mode = False
    
    def set_debug_mode(self, debug=True):
        """Enable or disable debug mode."""
        self.debug_mode = debug
        self.mark_detector.debug_mode = debug
    
    def extract_frames(self, image, frames_per_row=None):
        """
        Extract frames from a contact sheet using registration marks.
        
        Args:
            image: Input image (BGR format from OpenCV)
            frames_per_row: Optional hint for number of frames per row
            
        Returns:
            List of extracted frame images
        """
        # Check if image is valid
        if image is None or image.size == 0 or len(image.shape) < 2:
            print("Error: Invalid input image for frame extraction")
            return []
            
        # Create a copy for visualization if debug mode is on
        debug_img = image.copy() if self.debug_mode else None
        
        # Get image dimensions
        img_height, img_width = image.shape[:2]
        
        # Detect registration marks
        marks = self.mark_detector.detect_marks(image)
        
        if len(marks) < 8:  # Need at least 8 marks to extract frames reliably
            print(f"Warning: Only {len(marks)} registration marks detected. Need at least 8.")
            return []
        
        # Find frames by identifying groups of 4 registration marks that form corners
        extracted_frames = self._extract_frames_from_corners(image, marks, debug_img)
        
        # If we couldn't extract frames using the corner approach, try the grid approach
        if not extracted_frames:
            print("Corner-based extraction failed. Trying grid-based approach...")
            # Analyze grid structure
            grid_analysis = self.mark_detector.analyze_grid_structure(marks, img_width, img_height)
            rows = grid_analysis["rows"]
            
            # Use provided frames_per_row if available, otherwise use estimated value
            if frames_per_row is None:
                frames_per_row = grid_analysis["estimated_frames_per_row"]
                print(f"Automatically detected {frames_per_row} frames per row")
            
            # Extract frames using the grid structure
            extracted_frames = self._extract_frames_from_grid(image, rows, debug_img)
        
        # If we still couldn't extract frames, try a more flexible approach
        if not extracted_frames:
            print("Grid-based extraction failed. Trying alternative approach...")
            extracted_frames = self._extract_frames_alternative(image, marks, debug_img)
        
        # Save debug image if in debug mode
        if self.debug_mode and debug_img is not None:
            cv2.imwrite("debug_extracted_frames.png", debug_img)
        
        print(f"Successfully extracted {len(extracted_frames)} frames")
        return extracted_frames
    
    def _extract_frames_from_corners(self, image, marks, debug_img=None):
        """
        Extract frames by identifying groups of 4 registration marks that form corners.
        This is the most accurate approach when registration marks are clearly visible.
        
        Args:
            image: Input image
            marks: Detected registration marks
            debug_img: Optional image for debug visualization
            
        Returns:
            List of extracted frame images
        """
        extracted_frames = []
        img_height, img_width = image.shape[:2]
        
        # Limit the number of marks to process to prevent memory issues
        max_marks = 100  # Set a reasonable limit
        if len(marks) > max_marks:
            print(f"Warning: Limiting marks from {len(marks)} to {max_marks} to prevent memory issues")
            marks = marks[:max_marks]
        
        # Convert marks to numpy array for easier manipulation
        marks_array = np.array(marks)
        
        # Calculate pairwise distances between all marks
        dist_matrix = distance.cdist(marks_array, marks_array)
        
        # Find the median distance between adjacent marks (approximates frame width/height)
        # We'll use this to determine which marks might form a frame
        sorted_distances = np.sort(dist_matrix.flatten())
        # Remove zeros (self-distances)
        sorted_distances = sorted_distances[sorted_distances > 0]
        
        if len(sorted_distances) == 0:
            return []
        
        # Get the median of the smallest 25% of distances
        # This should approximate the distance between adjacent marks
        cutoff_idx = max(1, int(len(sorted_distances) * 0.25))
        median_small_distance = np.median(sorted_distances[:cutoff_idx])
        
        # Get the median of distances in the middle range (25% to 75%)
        # This should approximate the distance between marks that form a frame
        mid_start = int(len(sorted_distances) * 0.25)
        mid_end = int(len(sorted_distances) * 0.75)
        median_frame_distance = np.median(sorted_distances[mid_start:mid_end])
        
        # Define distance thresholds for finding frame corners
        min_frame_size = median_frame_distance * 0.5
        max_frame_size = median_frame_distance * 2.0
        
        # Set a maximum number of frames to extract to prevent infinite loops
        max_frames = 100
        frame_count = 0
        
        # For each mark, try to find 3 other marks that could form a frame with it
        # Limit the number of marks to check to prevent excessive iterations
        max_marks_to_check = min(50, len(marks))
        
        for i in range(max_marks_to_check):
            # This mark will be the top-left corner
            top_left = marks[i]
            
            # Find potential top-right corners
            potential_top_right = []
            # Limit the number of marks to check for top-right corners
            max_tr_checks = min(20, len(marks))
            tr_checks = 0
            
            for j in range(len(marks)):
                if tr_checks >= max_tr_checks:
                    break
                    
                if i == j:
                    continue
                    
                tr_checks += 1
                    
                # Check if this mark is to the right and at similar y-coordinate
                dx = marks[j][0] - top_left[0]
                dy = marks[j][1] - top_left[1]
                
                if dx > min_frame_size and dx < max_frame_size and abs(dy) < median_small_distance * 2:
                    potential_top_right.append((j, marks[j]))
            
            # Limit the number of potential top-right corners to check
            max_potential = 5
            potential_top_right = potential_top_right[:max_potential]
            
            # For each potential top-right corner
            for j, top_right in potential_top_right:
                # Find potential bottom-left corners
                potential_bottom_left = []
                # Limit the number of marks to check for bottom-left corners
                max_bl_checks = min(20, len(marks))
                bl_checks = 0
                
                for k in range(len(marks)):
                    if bl_checks >= max_bl_checks:
                        break
                        
                    if i == k or j == k:
                        continue
                        
                    bl_checks += 1
                        
                    # Check if this mark is below and at similar x-coordinate
                    dx = marks[k][0] - top_left[0]
                    dy = marks[k][1] - top_left[1]
                    
                    if dy > min_frame_size and dy < max_frame_size and abs(dx) < median_small_distance * 2:
                        potential_bottom_left.append((k, marks[k]))
                
                # Limit the number of potential bottom-left corners to check
                max_potential = 5
                potential_bottom_left = potential_bottom_left[:max_potential]
                
                # For each potential bottom-left corner
                for k, bottom_left in potential_bottom_left:
                    # Find potential bottom-right corners
                    # Limit the number of marks to check to prevent excessive iterations
                    max_checks = 20
                    checks = 0
                    
                    for l in range(len(marks)):
                        if checks >= max_checks:
                            break
                            
                        checks += 1
                        
                        if i == l or j == l or k == l:
                            continue
                            
                        # Check if this mark is to the right of bottom-left and below top-right
                        dx1 = marks[l][0] - bottom_left[0]
                        dy1 = marks[l][1] - top_right[1]
                        
                        # Also check if it forms a roughly rectangular shape
                        dx2 = abs((marks[l][0] - bottom_left[0]) - (top_right[0] - top_left[0]))
                        dy2 = abs((marks[l][1] - top_right[1]) - (bottom_left[1] - top_left[1]))
                        
                        if (dx1 > min_frame_size and dx1 < max_frame_size and 
                            dy1 > min_frame_size and dy1 < max_frame_size and 
                            dx2 < median_small_distance * 3 and dy2 < median_small_distance * 3):
                            # We found a potential frame!
                            bottom_right = marks[l]
                            
                            # Calculate frame coordinates with padding
                            padding = 5
                            x = top_left[0] + padding
                            y = top_left[1] + padding
                            w = bottom_right[0] - top_left[0] - 2 * padding
                            h = bottom_right[1] - top_left[1] - 2 * padding
                            
                            # Skip invalid frames
                            if w <= 20 or h <= 20 or x < 0 or y < 0 or x + w > img_width or y + h > img_height:
                                continue
                            
                            try:
                                # Extract the frame
                                frame = image[y:y+h, x:x+w].copy()
                                
                                # Verify frame is valid
                                if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                                    continue
                                
                                # Add to results
                                extracted_frames.append(frame)
                                frame_count += 1
                                
                                # Draw on debug image
                                if self.debug_mode and debug_img is not None:
                                    cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                                    # Add frame number
                                    frame_num = len(extracted_frames)
                                    cv2.putText(debug_img, str(frame_num), (x+10, y+30), 
                                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                                    
                                # Check if we've reached the maximum number of frames
                                if frame_count >= max_frames:
                                    print(f"Reached maximum frame limit of {max_frames}")
                                    return extracted_frames
                            except Exception as e:
                                print(f"Error extracting frame: {str(e)}")
                                continue
        
        return extracted_frames
    
    def _extract_frames_from_grid(self, image, rows, debug_img=None):
        """
        Extract frames using the grid structure of registration marks.
        This is a fallback approach when corner detection fails.
        
        Args:
            image: Input image
            rows: List of rows of registration marks
            debug_img: Optional image for debug visualization
            
        Returns:
            List of extracted frame images
        """
        extracted_frames = []
        img_height, img_width = image.shape[:2]
        
        # Set a maximum number of frames to extract to prevent infinite loops
        max_frames = 100
        frame_count = 0
        
        # For each pair of adjacent rows
        for row_idx in range(len(rows) - 1):
            top_row = rows[row_idx]
            bottom_row = rows[row_idx + 1]
            
            # Skip if either row has too few marks
            if len(top_row) < 2 or len(bottom_row) < 2:
                continue
            
            # Process each potential frame in this row
            for col_idx in range(len(top_row) - 1):
                # Skip if we don't have enough marks in the bottom row
                if col_idx >= len(bottom_row) - 1:
                    continue
                
                # Get the four corners of this frame
                top_left = top_row[col_idx]
                top_right = top_row[col_idx + 1]
                bottom_left = bottom_row[col_idx]
                bottom_right = bottom_row[col_idx + 1]
                
                # Calculate frame coordinates with padding
                padding = 5
                x = top_left[0] + padding
                y = top_left[1] + padding
                w = top_right[0] - top_left[0] - 2 * padding
                h = bottom_left[1] - top_left[1] - 2 * padding
                
                # Skip invalid frames
                if w <= 20 or h <= 20 or x < 0 or y < 0 or x + w > img_width or y + h > img_height:
                    print(f"Skipping invalid frame at ({x},{y},{w},{h})")
                    continue
                
                try:
                    # Extract the frame
                    frame = image[y:y+h, x:x+w].copy()
                    
                    # Verify frame is valid
                    if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                        print(f"Skipping empty frame at ({x},{y},{w},{h})")
                        continue
                    
                    # Add to results
                    extracted_frames.append(frame)
                    frame_count += 1
                    
                    # Draw on debug image
                    if self.debug_mode and debug_img is not None:
                        cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        # Add frame number
                        frame_num = len(extracted_frames)
                        cv2.putText(debug_img, str(frame_num), (x+10, y+30), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    
                    # Check if we've reached the maximum number of frames
                    if frame_count >= max_frames:
                        print(f"Reached maximum frame limit of {max_frames}")
                        return extracted_frames
                except Exception as e:
                    print(f"Error extracting frame at ({x},{y},{w},{h}): {str(e)}")
                    continue
        
        return extracted_frames
    
    def _extract_frames_alternative(self, image, marks, debug_img=None):
        """
        Alternative frame extraction method when other approaches fail.
        Uses a more flexible approach to identify frames.
        
        Args:
            image: Input image
            marks: Detected registration marks
            debug_img: Optional image for debug visualization
            
        Returns:
            List of extracted frame images
        """
        img_height, img_width = image.shape[:2]
        extracted_frames = []
        
        # Set a maximum number of frames to extract to prevent infinite loops
        max_frames = 100
        frame_count = 0
        
        # If we have at least 4 marks, try to find quadrilaterals
        if len(marks) >= 4:
            # Limit the number of marks to process to prevent memory issues
            max_marks = 50  # Set a reasonable limit
            if len(marks) > max_marks:
                print(f"Warning: Limiting marks from {len(marks)} to {max_marks} to prevent memory issues")
                marks = marks[:max_marks]
            
            # Convert marks to numpy array
            points = np.array(marks)
            
            # Calculate pairwise distances
            dist_matrix = distance.cdist(points, points)
            
            # Find potential frame corners
            for i in range(len(points)):
                # Get the 3 closest points to this point
                distances = dist_matrix[i]
                closest_indices = np.argsort(distances)[1:4]  # Skip self (index 0)
                
                if len(closest_indices) == 3:
                    # These 4 points (current point + 3 closest) might form a frame
                    quad_points = [points[i]]
                    for idx in closest_indices:
                        quad_points.append(points[idx])
                    
                    # Order points in clockwise order
                    quad_points = np.array(quad_points)
                    center = np.mean(quad_points, axis=0)
                    
                    # Sort points by their angle from center
                    angles = np.arctan2(quad_points[:, 1] - center[1], 
                                       quad_points[:, 0] - center[0])
                    sorted_indices = np.argsort(angles)
                    ordered_quad = quad_points[sorted_indices]
                    
                    # Check if this quadrilateral is a valid frame
                    # It should be roughly rectangular and of reasonable size
                    min_x = np.min(ordered_quad[:, 0])
                    max_x = np.max(ordered_quad[:, 0])
                    min_y = np.min(ordered_quad[:, 1])
                    max_y = np.max(ordered_quad[:, 1])
                    
                    width = max_x - min_x
                    height = max_y - min_y
                    
                    # Skip if too small or too large
                    if width < 30 or height < 30 or width > img_width*0.9 or height > img_height*0.9:
                        continue
                    
                    try:
                        # Extract frame with some padding
                        padding = 5
                        x = max(0, int(min_x) + padding)
                        y = max(0, int(min_y) + padding)
                        w = min(img_width - x, int(width) - 2 * padding)
                        h = min(img_height - y, int(height) - 2 * padding)
                        
                        if w <= 0 or h <= 0:
                            continue
                        
                        frame = image[y:y+h, x:x+w].copy()
                        
                        # Verify frame is valid
                        if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                            continue
                            
                        extracted_frames.append(frame)
                        frame_count += 1
                        
                        # Draw on debug image
                        if self.debug_mode and debug_img is not None:
                            cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                            # Add frame number
                            frame_num = len(extracted_frames)
                            cv2.putText(debug_img, str(frame_num), (x+10, y+30), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        
                        # Check if we've reached the maximum number of frames
                        if frame_count >= max_frames:
                            print(f"Reached maximum frame limit of {max_frames}")
                            return extracted_frames
                    except Exception as e:
                        print(f"Error in alternative extraction: {str(e)}")
                        continue
        
        # If still no frames, try a simple grid-based approach
        if not extracted_frames and len(marks) > 0:
            print("Trying simple grid-based approach...")
            # Find the bounding box of all marks
            x_coords = [m[0] for m in marks]
            y_coords = [m[1] for m in marks]
            min_x, max_x = min(x_coords), max(x_coords)
            min_y, max_y = min(y_coords), max(y_coords)
            
            # Extract a single frame from the center of the marks
            padding = 20
            x = max(0, min_x + padding)
            y = max(0, min_y + padding)
            w = min(img_width - x, max_x - min_x - 2 * padding)
            h = min(img_height - y, max_y - min_y - 2 * padding)
            
            if w > 30 and h > 30:
                try:
                    frame = image[y:y+h, x:x+w].copy()
                    if frame is not None and frame.size > 0 and frame.shape[0] > 0 and frame.shape[1] > 0:
                        extracted_frames.append(frame)
                        
                        # Draw on debug image
                        if self.debug_mode and debug_img is not None:
                            cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                            # Add frame number
                            frame_num = len(extracted_frames)
                            cv2.putText(debug_img, str(frame_num), (x+10, y+30), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                except Exception as e:
                    print(f"Error in simple grid extraction: {str(e)}")
        
        return extracted_frames

In [8]:
class AnimationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Animation Extractor")
        
        # Set up variables
        self.contact_sheet = None
        self.extracted_frames = []
        self.current_frame_index = 0
        self.is_playing = False
        self.play_after_id = None
        
        # Create UI variables
        self.status_var = tk.StringVar(value="Ready")
        self.frame_rate = tk.DoubleVar(value=12.0)  # Default 12 FPS
        self.frames_per_row = tk.IntVar(value=4)  # Default 4 frames per row
        self.extraction_mode = tk.StringVar(value="auto")  # Default to auto mode
        self.registration_mark_threshold = tk.IntVar(value=70)  # Default 70%
        self.debug_mode = tk.BooleanVar(value=False)  # Debug mode toggle
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create left panel for controls
        control_frame = ttk.Frame(main_frame, padding="5", width=200)
        control_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # Create right panel for image display
        display_frame = ttk.Frame(main_frame)
        display_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Create canvas for image display
        self.canvas = tk.Canvas(display_frame, bg="#f0f0f0", highlightthickness=0)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Input section
        input_frame = ttk.LabelFrame(control_frame, text="Input", padding="5")
        input_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(input_frame, text="Load Contact Sheet", command=self.load_contact_sheet).pack(fill=tk.X)
        ttk.Button(input_frame, text="Extract Frames", command=self.extract_frames_from_registration_marks).pack(fill=tk.X, pady=(5, 0))
        
        # Extraction mode section
        mode_frame = ttk.LabelFrame(control_frame, text="Extraction Settings", padding="5")
        mode_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Mode selection
        ttk.Label(mode_frame, text="Mode:").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="Auto", variable=self.extraction_mode, value="auto").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="Digital", variable=self.extraction_mode, value="digital").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="RISO Print", variable=self.extraction_mode, value="riso").pack(anchor=tk.W)
        
        # Frames per row setting
        ttk.Label(mode_frame, text="Frames Per Row:").pack(anchor=tk.W, pady=(5, 0))
        ttk.Spinbox(mode_frame, from_=1, to=20, textvariable=self.frames_per_row, width=5).pack(anchor=tk.W)
        
        # Registration mark threshold
        ttk.Label(mode_frame, text="Registration Mark Threshold (%):").pack(anchor=tk.W, pady=(5, 0))
        ttk.Spinbox(mode_frame, from_=10, to=90, textvariable=self.registration_mark_threshold, width=5).pack(anchor=tk.W)
        
        # Debug mode toggle
        ttk.Checkbutton(mode_frame, text="Debug Mode", variable=self.debug_mode).pack(anchor=tk.W, pady=(5, 0))
        
        # Extract button
        ttk.Button(mode_frame, text="Extract Frames", command=self.extract_frames_from_registration_marks).pack(fill=tk.X, pady=(5, 0))
        
        # Playback controls
        playback_frame = ttk.LabelFrame(control_frame, text="Playback", padding="5")
        playback_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Frame rate control
        ttk.Label(playback_frame, text="Frame Rate (FPS):").pack(anchor=tk.W)
        ttk.Spinbox(playback_frame, from_=1, to=60, textvariable=self.frame_rate, width=5).pack(anchor=tk.W, pady=(0, 5))
        
        # Playback buttons
        button_frame = ttk.Frame(playback_frame)
        button_frame.pack(fill=tk.X)
        
        ttk.Button(button_frame, text="⏮", width=3, command=self.go_to_first_frame).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏪", width=3, command=self.previous_frame).pack(side=tk.LEFT)
        self.play_button = ttk.Button(button_frame, text="▶", width=3, command=self.toggle_playback)
        self.play_button.pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏩", width=3, command=self.next_frame).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏭", width=3, command=self.go_to_last_frame).pack(side=tk.LEFT)
        
        # Export section
        export_frame = ttk.LabelFrame(control_frame, text="Export", padding="5")
        export_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(export_frame, text="Export as GIF", command=self.export_as_gif).pack(fill=tk.X)
        ttk.Button(export_frame, text="Export as MP4", command=self.export_as_mp4).pack(fill=tk.X, pady=(5, 0))
        ttk.Button(export_frame, text="Export Frames", command=self.export_frames).pack(fill=tk.X, pady=(5, 0))
        
        # Status bar
        status_bar = ttk.Label(root, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Bind keyboard shortcuts
        self.root.bind("<Left>", lambda e: self.previous_frame())
        self.root.bind("<Right>", lambda e: self.next_frame())
        self.root.bind("<space>", lambda e: self.toggle_playback())
        
        # Initialize the display
        self.display_placeholder()
    
    def display_placeholder(self):
        """Display a placeholder message when no image is loaded"""
        self.canvas.delete("all")
        self.canvas.create_text(
            self.canvas.winfo_width() // 2 or 200,
            self.canvas.winfo_height() // 2 or 150,
            text="Load a contact sheet or animation frames to begin",
            fill="#666666",
            font=("Arial", 12)
        )
    
    def load_contact_sheet(self):
        """Load a contact sheet image"""
        # Use standard file dialog for all platforms to avoid issues
        file_path = filedialog.askopenfilename(
            title="Select Contact Sheet",
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.tif *.tiff"),
                ("PDF files", "*.pdf"),
                ("All files", "*.*")
            ]
        )
        
        if not file_path:
            return
        
        try:
            # Handle PDF files
            if file_path.lower().endswith(".pdf"):
                # Convert first page of PDF to image using a temporary file
                with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_file:
                    temp_path = temp_file.name
                
                # Use pdftoppm to convert PDF to PNG
                subprocess.run(["pdftoppm", "-png", "-singlefile", file_path, temp_path[:-4]])
                
                # Load the converted image
                self.contact_sheet = Image.open(temp_path)
                
                # Clean up the temporary file
                os.unlink(temp_path)
            else:
                # Load regular image file
                self.contact_sheet = Image.open(file_path)
            
            # Display the contact sheet
            self.display_contact_sheet()
            
            # Update status
            self.status_var.set(f"Loaded contact sheet: {os.path.basename(file_path)}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error loading file: {str(e)}")
    
    def display_contact_sheet(self):
        """Display the loaded contact sheet"""
        if self.contact_sheet is None:
            self.display_placeholder()
            return
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        # If canvas hasn't been drawn yet, use default dimensions
        if canvas_width <= 1 or canvas_height <= 1:
            canvas_width = 600
            canvas_height = 400
        
        # Calculate scaling to fit the canvas
        img_width, img_height = self.contact_sheet.size
        scale = min(canvas_width / img_width, canvas_height / img_height)
        
        # Resize the image to fit the canvas
        new_width = int(img_width * scale)
        new_height = int(img_height * scale)
        resized_img = self.contact_sheet.resize((new_width, new_height), Image.LANCZOS)
        
        # Convert to PhotoImage for display
        self.tk_image = ImageTk.PhotoImage(resized_img)
        
        # Display on canvas
        self.canvas.delete("all")
        self.canvas.create_image(
            canvas_width // 2,
            canvas_height // 2,
            image=self.tk_image,
            anchor=tk.CENTER
        )
    
    def extract_frames_from_registration_marks(self):
        """
        Extract frames from a contact sheet using improved registration mark detection.
        """
        if self.contact_sheet is None:
            messagebox.showwarning("Warning", "No contact sheet loaded")
            return
                
        try:
            # Update status
            self.status_var.set("Extracting frames... This may take a moment.")
            self.root.update()
            
            # Convert PIL Image to OpenCV format
            cv_image = cv2.cvtColor(np.array(self.contact_sheet), cv2.COLOR_RGB2BGR)
            
            # Create frame extractor
            extractor = FrameExtractor()
            
            # Set debug mode if needed
            extractor.set_debug_mode(self.debug_mode.get())
            
            # Extract frames using the frames_per_row parameter as a hint
            frames = extractor.extract_frames(cv_image, self.frames_per_row.get())
            
            # Clean up to free memory
            del cv_image
            gc.collect()
            
            if not frames:
                messagebox.showwarning("Warning", "No frames could be extracted")
                return
            
            # Convert OpenCV frames back to PIL format
            pil_frames = []
            for frame in frames:
                # Verify frame is valid before conversion
                if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                    print("Skipping invalid frame during conversion")
                    continue
                    
                try:
                    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    pil_frames.append(pil_frame)
                    
                    # Clean up to free memory
                    del frame
                    gc.collect()
                except Exception as e:
                    print(f"Error converting frame: {str(e)}")
                    continue
            
            # Clean up to free memory
            del frames
            gc.collect()
            
            # Update the extracted frames
            self.extracted_frames = pil_frames
            self.current_frame_index = 0
            
            # Display the first frame if available
            if self.extracted_frames:
                self.display_current_frame()
                # Update status
                self.status_var.set(f"Successfully extracted {len(pil_frames)} frames")
            else:
                messagebox.showwarning("Warning", "No valid frames could be extracted")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error extracting frames: {str(e)}")
            import traceback
            traceback.print_exc()
    
    def display_current_frame(self):
        """Display the current frame"""
        if not self.extracted_frames:
            self.display_placeholder()
            return
        
        # Get the current frame
        frame = self.extracted_frames[self.current_frame_index]
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        # If canvas hasn't been drawn yet, use default dimensions
        if canvas_width <= 1 or canvas_height <= 1:
            canvas_width = 600
            canvas_height = 400
        
        # Calculate scaling to fit the canvas
        img_width, img_height = frame.size
        scale = min(canvas_width / img_width, canvas_height / img_height)
        
        # Resize the image to fit the canvas
        new_width = int(img_width * scale)
        new_height = int(img_height * scale)
        resized_img = frame.resize((new_width, new_height), Image.LANCZOS)
        
        # Convert to PhotoImage for display
        self.tk_image = ImageTk.PhotoImage(resized_img)
        
        # Display on canvas
        self.canvas.delete("all")
        self.canvas.create_image(
            canvas_width // 2,
            canvas_height // 2,
            image=self.tk_image,
            anchor=tk.CENTER
        )
        
        # Display frame number
        self.canvas.create_text(
            10, 10,
            text=f"Frame {self.current_frame_index + 1}/{len(self.extracted_frames)}",
            fill="white",
            font=("Arial", 12),
            anchor=tk.NW
        )
        
        # Update status
        self.status_var.set(f"Displaying frame {self.current_frame_index + 1} of {len(self.extracted_frames)}")
    
    def next_frame(self):
        """Display the next frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = (self.current_frame_index + 1) % len(self.extracted_frames)
        self.display_current_frame()
    
    def previous_frame(self):
        """Display the previous frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = (self.current_frame_index - 1) % len(self.extracted_frames)
        self.display_current_frame()
    
    def go_to_first_frame(self):
        """Go to the first frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = 0
        self.display_current_frame()
    
    def go_to_last_frame(self):
        """Go to the last frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = len(self.extracted_frames) - 1
        self.display_current_frame()
    
    def toggle_playback(self):
        """Toggle animation playback"""
        if not self.extracted_frames:
            messagebox.showinfo("Info", "No frames to play")
            return
        
        # If there's only one frame, playback doesn't make sense
        if len(self.extracted_frames) <= 1:
            messagebox.showinfo("Info", "Animation requires multiple frames for playback")
            return
        
        # Toggle playback state
        self.is_playing = not self.is_playing
        
        # Update button text
        self.play_button.config(text="⏸" if self.is_playing else "▶")
        
        # Start or stop playback
        if self.is_playing:
            self.play_animation()
        else:
            # Cancel any scheduled playback
            if self.play_after_id:
                self.root.after_cancel(self.play_after_id)
                self.play_after_id = None
    
    def play_animation(self):
        """Play the animation"""
        if not self.is_playing or not self.extracted_frames:
            return
        
        try:
            # Display the current frame
            self.display_current_frame()
            
            # Move to the next frame
            self.current_frame_index = (self.current_frame_index + 1) % len(self.extracted_frames)
            
            # Calculate delay based on frame rate
            # Make sure frame rate is at least 1 FPS to avoid division by zero
            frame_rate = max(1.0, self.frame_rate.get())
            delay_ms = int(1000 / frame_rate)
            
            # Schedule the next frame
            self.play_after_id = self.root.after(delay_ms, self.play_animation)
            
        except Exception as e:
            print(f"Error during playback: {str(e)}")
            self.is_playing = False
            self.play_button.config(text="▶")
    
    def export_as_gif(self):
        """Export the animation as a GIF"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        try:
            # Use standard file dialog
            file_path = filedialog.asksaveasfilename(
                title="Save GIF Animation",
                defaultextension=".gif",
                filetypes=[("GIF files", "*.gif"), ("All files", "*.*")]
            )
            
            if not file_path:
                return
            
            # Ensure file has .gif extension
            if not file_path.lower().endswith(".gif"):
                file_path += ".gif"
            
            # Get export parameters
            fps = self.frame_rate.get()
            
            # Calculate frame duration in milliseconds
            duration = int(1000 / fps)
            
            # Update status
            self.status_var.set("Exporting GIF... This may take a moment.")
            self.root.update()
            
            # Save as GIF
            self.extracted_frames[0].save(
                file_path,
                save_all=True,
                append_images=self.extracted_frames[1:],
                optimize=False,
                duration=duration,
                loop=0  # 0 means loop forever
            )
            
            # Update status
            self.status_var.set(f"Exported GIF animation to {file_path}")
            
            # Show success message
            messagebox.showinfo("Success", f"Animation exported to {file_path}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error exporting GIF: {str(e)}")
    
    def export_as_mp4(self):
        """Export the animation as an MP4 video"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        try:
            # Use standard file dialog
            file_path = filedialog.asksaveasfilename(
                title="Save MP4 Video",
                defaultextension=".mp4",
                filetypes=[("MP4 files", "*.mp4"), ("All files", "*.*")]
            )
            
            if not file_path:
                return
            
            # Ensure file has .mp4 extension
            if not file_path.lower().endswith(".mp4"):
                file_path += ".mp4"
            
            # Get export parameters
            fps = self.frame_rate.get()
            
            # Get dimensions of first frame
            width, height = self.extracted_frames[0].size
            
            # Update status
            self.status_var.set("Exporting MP4... This may take a moment.")
            self.root.update()
            
            # Create a temporary directory for frame images
            with tempfile.TemporaryDirectory() as temp_dir:
                try:
                    # Save frames as temporary images
                    frame_paths = []
                    for i, frame in enumerate(self.extracted_frames):
                        frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
                        frame.save(frame_path)
                        frame_paths.append(frame_path)
                    
                    # Create video writer
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
                    video_writer = cv2.VideoWriter(file_path, fourcc, fps, (width, height))
                    
                    # Add each frame to the video
                    for frame_path in frame_paths:
                        img = cv2.imread(frame_path)
                        if img is not None and img.size > 0:
                            video_writer.write(img)
                        else:
                            print(f"Warning: Could not read frame from {frame_path}")
                    
                    # Release the video writer
                    video_writer.release()
                    
                    # Update status
                    self.status_var.set(f"Exported MP4 video to {file_path}")
                    
                    # Show success message
                    messagebox.showinfo("Success", f"Video exported to {file_path}")
                    
                except Exception as e:
                    raise Exception(f"Error creating video: {str(e)}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error exporting MP4: {str(e)}")
    
    def export_frames(self):
        """Export individual frames as image files"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        try:
            # Use standard folder dialog
            export_dir = filedialog.askdirectory(title="Select Export Directory")
            
            if not export_dir:
                return
            
            # Update status
            self.status_var.set("Exporting frames... This may take a moment.")
            self.root.update()
            
            # Export frames
            successful_exports = 0
            for i, frame in enumerate(self.extracted_frames):
                try:
                    frame_path = os.path.join(export_dir, f"frame_{i+1:04d}.png")
                    frame.save(frame_path)
                    successful_exports += 1
                except Exception as e:
                    print(f"Error saving frame {i+1}: {str(e)}")
            
            # Update status
            self.status_var.set(f"Exported {successful_exports} frames to {export_dir}")
            
            # Show success message
            messagebox.showinfo("Success", f"Exported {successful_exports} frames to {export_dir}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error exporting frames: {str(e)}")

In [9]:
# Create and run the application
root = tk.Tk()
app = AnimationApp(root)

# Set window size
root.geometry("800x600")

# Start the main loop
root.mainloop()

2025-03-25 16:19:38.777 Python[20213:1031748] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-25 16:19:38.777 Python[20213:1031748] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Corner-based extraction failed. Trying grid-based approach...
Skipping invalid frame at (198,114,-1,75)
Skipping invalid frame at (207,157,65,-2)
Skipping invalid frame at (381,143,14,14)
Skipping invalid frame at (405,133,8,46)
Skipping invalid frame at (423,147,19,57)
Skipping invalid frame at (452,157,19,7)
Skipping invalid frame at (481,141,24,15)
Skipping invalid frame at (515,115,16,74)
Skipping invalid frame at (541,156,11,13)
Skipping invalid frame at (562,162,6,27)
Skipping invalid frame at (578,128,12,38)
Skipping invalid frame at (735,161,15,37)
Skipping invalid frame at (838,115,19,60)
Skipping invalid frame at (867,141,13,31)
Skipping invalid frame at (890,139,9,43)
Skipping invalid frame at (909,153,11,14)
Skipping invalid frame at (1034,162,38,-5)
Skipping invalid frame at (234,199,14,45)
Skipping invalid frame at (258,165,10,88)
Skipping invalid frame at (278,196,41,12)
Skipping invalid frame at (371,189,64,20)
Skipping invalid frame at (486,174,20,38)
Skipping invalid 